# SEIRX model runs for test technology and screening frequency investigation

In [1]:
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

# custom functionality
from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
import data_creation_functions as dcf

## Parameters

### Simulation parameters and measures in place

In [2]:
with open('params/testing_strategy_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/testing_strategy_measures.json') as json_file:
    measures = json.load(json_file)


In [3]:
# paths for I/O
contact_network_src = '../data/contact_networks'
ensmbl_dst = '../data/simulation_results/ensembles/testing_strategy'

### Screening parameters

In [4]:
# test technologies (and test result turnover times) used in the
# different scenarios
test_types = ['same_day_antigen', 'same_day_LAMP', 'same_day_PCR',\
              'one_day_PCR', 'two_day_PCR']

# specifies, whether the index case will be introduced via an
# employee or a resident
index_cases = ['employee', 'resident']

# resident and employee streening intervals (in days)
e_screen_range = [2, 3, 7, None]
r_screen_range = [2, 3, 7, None]

# ensemble_size
N_runs = 5000

screening_params = [(i, j, k, l, N_runs)\
              for i in test_types \
              for j in index_cases \
              for k in e_screen_range \
              for l in r_screen_range]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 160 parameter combinations


In [5]:
def run(param_list):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''    

    # extract the simulation parameters from the parameter list
    test_type,index_case,e_screen_interval,r_screen_interval,N_runs = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, measures, simulation_params,
                contact_network_src, ensmbl_dst, test_type=test_type,
                index_case=index_case, e_screen_interval=e_screen_interval,
                r_screen_interval=r_screen_interval)

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results,index_case,test_type=test_type,
                                e_screen_interval=e_screen_interval,
                                r_screen_interval=r_screen_interval)

    return row

In [6]:
%%time
number_of_cores = psutil.cpu_count(logical=True) - 3
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
dst = '../data/simulation_results'
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['test_type', 'index_case',
              'resident_screen_interval', 'employee_screen_interval']
other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

results.to_csv(join(dst,'simulations_testing_strategy_{}.csv'\
                   .format(N_runs)), index=False)
results.head(3)

100%|██████████| 160/160 [1:51:16<00:00, 41.73s/it] 


CPU times: user 3.79 s, sys: 313 ms, total: 4.1 s
Wall time: 1h 51min 18s


,test_type,index_case,resident_screen_interval,employee_screen_interval,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,N_agents_0.90,N_agents_0.975,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,same_day_antigen,employee,2.0,2.0,53.0,53.0,53.0,53.0,53.0,53.0,...,1.365637,0.0,0.0,0.0,0.0,0.0,1.0,0.0654,0.0,0.257558
1,same_day_antigen,employee,3.0,2.0,53.0,53.0,53.0,53.0,53.0,53.0,...,1.785870,0.0,0.0,0.0,0.0,0.0,1.0,0.0742,0.0,0.288983
2,same_day_antigen,employee,2.0,3.0,53.0,53.0,53.0,53.0,53.0,53.0,...,1.816329,0.0,0.0,0.0,0.0,0.0,1.0,0.0724,0.0,0.287010
